# Combine Movie Functions

In [4]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from wikipedia import wikipedia
from wikifilm import wikifilm
from oscar import oscars
from razzies import razzies
from rollingstone import rollingstone
from BAFTA import BAFTA
from SAG import SAG
from filmsite import filmsite
from films101 import films101
from AACTA import AACTA
from flops import flops
from canada import Canada
from boxofficemojo import boxofficemojo
from rottentomatoes import rottentomatoes
from ultimatemovierankings import ultimatemovierankings
from combine import combine


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-03-02 11:46:00.583184


# Get/Parse/Merge/Process Rotten Tomatoes Data

In [11]:
%load_ext autoreload
%autoreload

comb = combine()
comb.setOscarData(oscars(wikipedia()), 10)
comb.setWikiFilmData(wikifilm(), 1)
comb.setRazziesData(razzies(), 1)
comb.setBAFTAData(BAFTA(), 1)
comb.setSAGData(SAG(), 1)
comb.setAACTAData(AACTA(), 1)
comb.setFilmsiteData(filmsite(), 1)
comb.setFilms101Data(films101(), 1)
comb.setFlopsData(flops(), 1)
comb.setCanadaData(Canada(), 1)
comb.setRollingStoneData(rollingstone(), 1)
comb.setRottenTomatoesData(rottentomatoes(), 90)
comb.setBoxOfficeMojoData(boxofficemojo(), 5e6)
comb.setUltimateMovieRankingsData(ultimatemovierankings(), 90)
comb.getData()
comb.saveCorrections(debug=True)
comb.mergeMovies(debug=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 67 Years of Ultimate Movie Rankings Movies
Found 119 Years of Films101 Movies
Found 61 Years of Canada Movies
Found 105 Years of Filmsite Movies
Found 98 Years of Wiki Film Movies
Found 47 Years of BAFTA Movies
Found 94 Years of Rotten Tomatoes Movies
Found 23 Years of Rolling Stone Movies
Found 25 Years of SAG Movies
Found 39 Years of Box Office Mojo Movies
Found 38 Years of Razzies Movies
Found 50 Years of AACTA Movies
Found 84 Years of Oscar Movies
Found 32 Years of Flops Movies
Found Data Between 1900 and 2018
Found 745 old corrections
Found 28 new corrections
Adding Crocodile Dundee
Adding Au Hasard Balthazar
Adding new corrections to Au Revoir Les Enfants
Adding new corrections to Bob le Flambeur
Adding new corrections to Chikamatsu Monogatari
Adding Cousin Cousine
Adding Diary Of A Chambermaid
Adding E.T. The Extra-Terrestrial
Adding Fall Of The House Of Usher
Adding Gerald McBoing-Boin

TypeError: unhashable type: 'dict'

# Find Corrections

In [2]:
from ioUtils import getFile
data = getFile("/Users/tgadfort/Documents/code/movies/combine/results/movies.json")
for movie,name in data.items():
    year  = movie[-5:-1]
    movie = movie[:-7]
    
    pos = movie.rfind("(")
    if pos == -1:
        continue        

    if pos > len(movie) - 15 and movie.find(",") != -1:
        fix = movie[:pos].strip()
        print("{0}:".format(fix))
        print(" - {0}".format(movie))

Walt Disney Treasures - The Chronological Donald, Volume One:
 - Walt Disney Treasures - The Chronological Donald, Volume One (1934 - 1941)
Walt Disney Treasures - The Chronological Donald, Volume Three:
 - Walt Disney Treasures - The Chronological Donald, Volume Three (1947 - 1950)
Aparajito:
 - Aparajito (1956, India)
Walt Disney Treasures - The Chronological Donald, Volume Four:
 - Walt Disney Treasures - The Chronological Donald, Volume Four (1951 - 1961)


In [16]:
from searchUtils import findNearest
from fsUtils import setFile
from ioUtils import saveFile

savename = setFile(comb.getDataDir(), "saved.yaml")
savedData = getFile(savename)

movies = comb.getCombinedMovies()

repData = {}
for corrMovie,corrs in savedData.items():
    for corr in corrs:
        try:
            repData[corr] = corrMovie
        except:
            raise ValueError(corr,corrMovie)
            
print("Found {0} movies".format(len(movies)))
print("Found {0} corrections".format(len(repData)))

Found 16616 movies
Found 939 corrections


In [21]:
movielist = {}
for moviename in movies.keys():
    name = moviename[:-7]
    year = moviename[-5:-1]
    try:
        year = int(year)
    except:
        raise ValueError(moviename)

    if repData.get(name) is not None:
        name = repData[name]

    if name.find("IMAX") != -1:
        continue
    if name.find("re-issue") != -1:
        continue        
        
    for y in range(year-2,year+3):
        if movielist.get(y) is None:
            movielist[y] = []
        movielist[y].append(name)

for year in movielist.keys():
    movielist[year] = set(movielist[year])

In [24]:
data   = {}
cutoff = 0.9

for year,movienames in movielist.items():
    print(year,len(movienames))
    
    for im,movie in enumerate(list(movienames)):
        if movie.endswith(' '):        
            if data.get(movie[:-1]) is None:
                print('\t',len(data),'\t',movie[:-1])
                data[movie[:-1]] = set()
                data[movie[:-1]].add(movie)
            tmpList = list(filter(lambda a: a != movie[:-1], list(movienames)))
            results = findNearest(movie[:-1], tmpList, 3, cutoff)
            if len(results) > 0:
                for value in results:
                    data[movie[:-1]].add(value)
        else:
            tmpList = list(filter(lambda a: a != movie, list(movienames)))    
            results = findNearest(movie, tmpList, 3, cutoff)
            if len(results) > 0:
                if data.get(movie) is None:
                    print('\t',len(data),'\t',movie)
                    data[movie] = set()
                for value in results:
                    data[movie].add(value)

        if len(data) > 300:
            break
        
for k in data.keys():
    data[k] = list(data[k])

            

savename = setFile(comb.getDataDir(), "corr.yaml")
saveFile(idata=data, ifile=savename, debug=True)

1898 8
1899 19
1900 25
1901 33
1902 42
1903 41
1904 38
1905 42
1906 41
1907 45
1908 64
1909 75
1910 93
1911 109
1912 130
1913 129
1914 153
1915 159
1916 165
1917 163
1918 181
1919 179
1920 196
1921 209
1922 221
	 0 	 Sherlock, Jr.
	 1 	 Sherlock Jr.
1923 236
1924 260
	 2 	 The Better 'Ole
	 3 	 The Better Ole
1925 277
1926 316
1927 368
1928 381
	 4 	 The Divorcee
	 5 	 The Divorcée
1929 399
1930 418
1931 442
1932 435
1933 460
	 6 	 Gold Diggers Of 1935
	 7 	 Gold Diggers Of 1933
1934 476
	 8 	 Rose Marie
	 9 	 Rose-Marie
1935 481
1936 471
1937 477
	 10 	 The Story Of The Last Chrysanthemum(s)
	 11 	 The Story Of The Last Chrysanthemum
1938 475
	 12 	 The Shop around The Corner
	 13 	 The Shop Around The Corner
1939 483
	 14 	 Tom, Dick And Harry
	 15 	 The Devil & Daniel Webster
	 16 	 The Devil And Daniel Webster
	 17 	 Tom, Dick, And Harry
1940 483
1941 505
1942 514
1943 528
	 18 	 The Woman In The Window
	 19 	 Woman In The Window
1944 508
	 20 	 Rome, Open City
	 21 	 Rome Open Cit

	 248 	 National Treasure 2 - Book Of Secrets
	 249 	 The King Of Kong A Fistful Of Quarters
	 250 	 Sweeney Todd - The Demon Barber Of Fleet Street
2006 1617
	 251 	 Transporter 3
	 252 	 The Sisterhood Of The Traveling Pants 2
	 253 	 RocknRolla
	 254 	 Transporter 2
	 255 	 The Sisterhood Of The Traveling Pants
	 256 	 RockNRolla
	 257 	 Saw V
	 258 	 Man On Wire
	 259 	 Man On Fire
2007 1608
	 260 	 The Messengers
	 261 	 500 Days Of Summer
	 262 	 The Bad Lieutenant Port Of Call: New Orleans
	 263 	 Halloween 2
	 264 	 The Pink Panther 2
	 265 	 The Pink Panther
	 266 	 Yoo Hoo, Mrs. Goldberg
	 267 	 Yoo-hoo, Mrs. Goldberg
	 268 	 (500) Days Of Summer
	 269 	 Halloween
	 270 	 Bad Lieutenant Port Of Call New Orleans
	 271 	 Red Riding: 1980
	 272 	 Saw VI
	 273 	 Monsters vs Aliens
	 274 	 Red Riding: 1983
	 275 	 The Messenger
	 276 	 Monsters Vs. Aliens
2008 1590
	 277 	 Sex And The City
	 278 	 Why Did I Get Married?
	 279 	 Waiting For Superman
	 280 	 Paranormal Activity
	 28